<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula10d_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5055k      0  0:00:16  0:00:16 --:--:-- 10.8M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [2]:
import os, pathlib, shutil, random
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
train_pos_dir = train_dir / 'pos'
train_neg_dir = train_dir / 'neg'
val_pos_dir = val_dir / 'pos'
val_neg_dir = val_dir / 'neg'
for category in ('neg', 'pos'):
  new_dir = val_dir / category
  if not new_dir.exists():
    os.makedirs(new_dir)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
      shutil.move(train_dir / category / fname,
                  val_dir / category / fname)


In [3]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [4]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'This was a delightful presentation. Hemo (blood) as a Greek god was so well played by the animation with vanity, arrogance, snobbish superiority and innocent wonder. The quote (or scene) I recall vividly is when Hemo tires of "all this plumbing ... you haven\'t learned my secrets at all" and threatens to storm out, the Scientist answers him in a single word "Thalassa" -- salt water which horrifies the Fiction Writer but mollifies Hemo and segues so neatly into the chemical aspects of blood. <br /><br />Such a splendid blend of entertainment and information make this a classic as fresh and engrossing today as the day it was released. Stimulating the interest and imagination is fundamental to teaching kids to love learning.', shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [19]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    ngrams=3,
    max_tokens=20000,
    output_mode='tf_idf')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)
vect_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
vect_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
vect_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [20]:
for inputs, targets in vect_train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[311.57568     0.6975004   2.8452163 ...   0.          0.
   0.       ], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [21]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation='relu')(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation='sigmoid')(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [22]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("model.keras",
                                    save_best_only=True)
]
history = model.fit(vect_train_ds.cache(),
                    validation_data=vect_val_ds.cache(),
                    epochs=10,
                    callbacks=callbacks)
model = keras.models.load_model("model.keras")
test_loss, test_acc = model.evaluate(vect_test_ds)
print(f"Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}")

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.5856 - loss: 1.1344 - val_accuracy: 0.8762 - val_loss: 0.3754
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6790 - loss: 0.5317 - val_accuracy: 0.8768 - val_loss: 0.3248
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7122 - loss: 0.5021 - val_accuracy: 0.8734 - val_loss: 0.3227
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7532 - loss: 0.4644 - val_accuracy: 0.8658 - val_loss: 0.3463
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7643 - loss: 0.4462 - val_accuracy: 0.8578 - val_loss: 0.3186
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7817 - loss: 0.4358 - val_accuracy: 0.8800 - val_loss: 0.3058
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8103 - loss: 0.4121 - val_accuracy: 0.8676 - val_loss: 0.3128
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8391 - loss: 0.3775 - val_accuracy: 